# RNN with NodeAnalysis


In [1]:
from __future__ import print_function

import collections

import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf

from tensorflow.keras import layers

import numpy as np
import tensorflow as tf 
import random
import time

from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec, KeyedVectors, Phrases

from nodeanalysis.NodeAnalysis import NodeAnalysisCallback
from nodeanalysis.nodes.EmptyNode import EmptyNode


## Pre-Processing

In [2]:
start_time = time.time()
def elapsed(sec):
    if sec<60:
        return str(sec) + " sec"
    elif sec<(60*60):
        return str(sec/60) + " min"
    else:
        return str(sec/(60*60)) + " hr"


# Target log path
logs_path = 'tmp'
#writer = tf.summary.FileWriter(logs_path)#this line was change to the line below by YKU on 05/18/20
#writer = tf.train.SummaryWriter(logs_path)
# Text file containing words for training
#training_file = '..\outputFiles\WordCountBytecodeHex.txt' the line was changed to the following line by YKU on 05/18/20
#training_file = 'outputFiles/WordCountBytecodeHex.txt'
training_file = 'inputFiles/The_D_of_I.txt'
#training_file = 'inputFiles/QuickBrownFox.txt'
#training_file = 'inputFiles/TwoSentences.txt'
#training_file = 'inputFiles/russian_text.txt'
def read_data(fname):
    with open(fname) as f:
        content = f.readlines()
    content = [x.strip() for x in content]
    content = [word for i in range(len(content)) for word in content[i].split()]
    content = np.array(content)
    return content

training_data = read_data(training_file)
print("Loaded training data...")

def build_dataset(words):
    count = collections.Counter(words).most_common()
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return dictionary, reverse_dictionary

dictionary, reverse_dictionary = build_dataset(training_data)
vocab_size = len(dictionary)

# Parameters
learning_rate = 0.001
training_iters = 50000
display_step = 1000
# public static --> void main...
# basic words to be used for prediction, coming soon......
n_input = 3

# number of units in RNN cell
n_hidden = 512

Loaded training data...


## Pre-Processing that I Added

In [3]:
def encode(data, dictt):
    return [dictt[data[x]] for x in range(0,len(data))]

def group(data, num):
    result = []
    for i in range(0,len(data)-num+1):
        result.append([data[i+x] for x in range(0,num)])
    return result

def expand(groups, size):
    result = []
    while len(result) < size:
        for x in range(0,len(groups)):
            result.append(groups[x])
            if len(result) >= size:
                break
    return result

def seperate(groups, index):
    x = []
    y = []
    for a in range(0,len(groups)):
        x.append(groups[a][:index])
        y.append(groups[a][index:])
    return x, y

def batches(data, dictt, input_size, batch_size):
    x, y = seperate(expand(group(encode(data,dictt),input_size+1),batch_size),input_size)
    return np.array(x), np.array(y)

## Word2Vector

In [4]:
w2v_vector_dim = 25
w2v_epochs = 50
def w2v_model(data,num):
    global w2v_vector_dim
    global w2v_epochs

    w2v_data = group(data,num)
    model = Word2Vec(w2v_data, size=w2v_vector_dim, window=5, min_count=1, workers=4, iter=w2v_epochs)

    return model

print("starting to train word 2 vector model...")
w2v_input_size = 10
w2v = w2v_model(training_data, w2v_input_size)
w2v_dict = w2v.wv
print("word 2 vector model done training...")

starting to train word 2 vector model...
word 2 vector model done training...


## Keras Model

In [5]:
model = tf.keras.Sequential()
# Add an Embedding layer expecting input vocab of size 1000, and
# output embedding dimension of size 64.
# model.add(layers.Embedding(vocab_size, 64, input_length=n_input)) <-- replaced by w2v
# model.add(layers.Dense(64, input_shape=(3,10)))
# Add a LSTM layer with 128 internal units.
# model.add(layers.LSTM(128))
new_shape = (n_input, w2v_vector_dim,)
# model.add(layers.Dense(64, input_shape=new_shape))


rnn_cell = tf.keras.layers.StackedRNNCells([tf.keras.layers.LSTMCell(n_hidden),tf.keras.layers.LSTMCell(n_hidden)])
# model.add(layers.RNN(rnn_cell, input_length=n_input))
model.add(layers.RNN(rnn_cell, input_shape=new_shape))
# Add a Dense layer with 10 units.
model.add(layers.Dense(len(dictionary)))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rnn (RNN)                    (None, 512)               3201024   
_________________________________________________________________
dense (Dense)                (None, 627)               321651    
Total params: 3,522,675
Trainable params: 3,522,675
Non-trainable params: 0
_________________________________________________________________


## Prepare Batches

In [6]:
epochs = 50
batch_size_param = 1
batch_size = 1000*batch_size_param

dictt, reverse_dictt = build_dataset(training_data)
x_train, _ = batches(training_data, w2v_dict, n_input, batch_size)
(x_val, _) = batches(training_data, w2v_dict, n_input, batch_size*0.25)
(x_val2, _) = batches(training_data, dictt, n_input, batch_size*0.25)
_, y_train = batches(training_data, dictt, n_input, batch_size)
_, y_val = batches(training_data, dictt, n_input, batch_size*0.25)

print(x_train.shape)
print(y_train.shape)

model.compile(optimizer=tf.keras.optimizers.RMSprop(),  # Optimizer
              # Loss function to minimize
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              # List of metrics to monitor
              metrics=['sparse_categorical_accuracy'])

print('# Fit model on training data')

(1000, 3, 25)
(1000, 1)
# Fit model on training data


## Tensorboard Callback

In [7]:
logs_path = "tb25"
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logs_path, histogram_freq=1)

## Nodeanalysis Callback

In [8]:
nac = NodeAnalysisCallback()

## Model Training

In [ ]:
history = model.fit(x_train, y_train,
                    batch_size=batch_size_param,
                    epochs= 5, # epochs,
                    # We pass some validation for
                    # monitoring validation loss and metrics
                    # at the end of each epoch
                    #validation_data=(x_val, y_val),
                    callbacks=[tensorboard_callback, nac])

Epoch 1/5
1000/1000 [==============================] - 60s 60ms/step - loss: 5.9456 - sparse_categorical_accuracy: 0.0550
Epoch 2/5
1000/1000 [==============================] - 61s 61ms/step - loss: 5.5736 - sparse_categorical_accuracy: 0.0630 2s - loss: 5.5832
Epoch 3/5
 435/1000 [============>.................] - ETA: 32s - loss: 5.4379 - sparse_categorical_accuracy: 0.08

## Show Test Results

In [ ]:
predictions = model.predict(x_val)

for x in range(0,len(predictions)):
    prediction = predictions[x]
    max = 0
    for i in range(0,len(prediction)):
        if prediction[i] > prediction[max]:
            max = i
    print(f'input: { [reverse_dictt[x_val2[x][i]] for i in range(0,len(x_val[x]))] } pred: {reverse_dictt[max]} --- actual: {[reverse_dictt[y_val[x][i]] for i in range(0,len(y_val[x]))] }')


## Results

### Accuracy

In [ ]:
plt.plot(history.history['sparse_categorical_accuracy'])
# plt.plot(history.history['val_sparse_categorical_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train'], loc='upper left')
plt.show()

### Loss


In [ ]:
plt.title('model loss')
plt.plot(history.history['loss'])
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()


## Duration


In [ ]:
end_time = time.time()
print("duration: ",elapsed(end_time - start_time))

## NodeAnalysis

In [ ]:
class LSTMGroup(EmptyNode):
    def __init__(self):
        super(LSTMGroup, self).__init__()
        self.units = None
        self.kernel = None
        self.recurrent_kernel = None
        self.bias_all = None
        
        self.kernel_dict = None
        self.recurrent_kernel_dict = None
        self.bias_dict = None
        
        # weights type
        self.KERNEL = "KERNEL"
        self.RE_KERNEL = "RECURRENT KERNEL"
    
    def report(self):
        super(LSTMGroup, self).report()
        
    def getSimpleNeuronHistory(self, weights_type, node_index, epochs):
        h = [getSimpleNeuron(weights_type, node_index) for e in range(epochs)]
        return h
        
    
    def getSimpleNeuron(self, weights_type, node_index):
        node = SimpleNeuron()
        
        node.name = f"{self.name} -- {weights_type} : {node_index}"
        node.model = self.model
        node.epoch = self.epoch
        
        node.layer = self.layer
        node.layer_in = self.layer_in
        node.layer_out = self.layer_out
        
        node.layer_index = self.layer_index
        node.node_index = [self.node_index, node_index]
        node.nac = self.nac
        
        node.epochs = len(nac.weights[self.layer.name])
        
        if weights_type == self.KERNEL:
            node.weight = self.kernel

        if weights_type == self.RE_KERNEL:
            node.weight = self.recurrent_kernel
            
        for index in node_index:
            node.weight = node.weight[index]
        
        node.bias = self.bias_all[node_index[-1]]
        
        return node
        
        
    def get(self, nac, layer_index, node_index,  epoch=-1):
        super(LSTMGroup, self).get(nac, layer_index, node_index, epoch=epoch)
        
        W = nac.weights[self.layer.name][epoch][node_index*3]
        U = nac.weights[self.layer.name][epoch][node_index*3 + 1]
        b = nac.weights[self.layer.name][epoch][node_index*3 + 2]
        
        self.kernel = W
        self.recurrent_kernel = U
        self.bias_all = b

        units = int(int(self.kernel.shape[1])/4)
        self.units = units
        
        # input - forget - cell state - output
        W_i = W[:, :units]
        W_f = W[:, units: units * 2]
        W_c = W[:, units * 2: units * 3]
        W_o = W[:, units * 3:]
        
        self.kernel_dict = {
            "input" : W_i,
            "forget" : W_f,
            "cell_state" : W_c,
            "output" : W_o
        }

        U_i = U[:, :units]
        U_f = U[:, units: units * 2]
        U_c = U[:, units * 2: units * 3]
        U_o = U[:, units * 3:]
        
        self.recurrent_kernel_dict = {
            "input" : U_i,
            "forget" : U_f,
            "cell_state" : U_c,
            "output" : U_o 
        }

        b_i = b[:units]
        b_f = b[units: units * 2]
        b_c = b[units * 2: units * 3]
        b_o = b[units * 3:]
        
        self.bias_dict = {
            "input" : b_i,
            "forget" : b_f,
            "cell_state" : b_c,
            "output" : b_o 
        }
        
        return self

In [ ]:
group = LSTMGroup()

In [ ]:
group.get(nac,0,0)
group.getSimpleNeuron(group.KERNEL, [0,0]).weight
# print(group.get(nac, 0, 0).getNode(group.KERNEL, [0,0]).weight)

In [ ]:
h = group.getSimpleNeuronHistory(group.KERNEL, [0,0], group.epochs)